# Programmation d'un modèle de langue n-gram

# Import des packages

In [8]:
%load_ext autoreload
%autoreload 2

import numpy as np

from ngramoptimized import NGramModel

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Traitement des données d'apprentissage

On utilise ici l'extrait assez faible (15 Mo) disponible sur le drive de Benjamin.

# Tests
## Greedy prediction

In [9]:
probas = np.array([1, 2, 3, 4])
probas = probas / sum(probas)
probas**2 / sum(probas**2)

array([0.03333333, 0.13333333, 0.3       , 0.53333333])

In [10]:
with open("fr_wikipedia_sample.txt","r",encoding="utf8") as file:
    corpus=file.read()
corpus = corpus.lower()
seqcorpus = corpus.split(' ')

In [7]:
for i in range(2,6):
    # print("Résultats pour",i,"gram")
    model = NGramModel(i)
    model.fit(seqcorpus)
    print("Predicting...")
    text = " ".join(model.generate_greedy(25,". elle est la plus"))
    print(text)
    print("")

Fitting 2-gram model on vocabulary of size 133657.
Predicting...
. elle est la plus de la première fois , le plus de la première fois , le plus de la première fois , le plus de la première fois

Fitting 3-gram model on vocabulary of size 133657.
Predicting...
. elle est la plus grande partie de la population de l' état de la population de l' état de la population de l' état de la population de l'

Fitting 4-gram model on vocabulary of size 133657.
Predicting...
. elle est la plus grande partie de la population , dont la plus grande partie de la population , dont la plus grande partie de la population , dont

Fitting 5-gram model on vocabulary of size 133657.
Predicting...
. elle est la plus grande ville du pays .
une capitale ( du latin , tête ) est une ville belge dans la région flamande , chef-lieu de la province



## Sample Method

In [12]:
for i in range(2,6):
    # print("Résultats pour",i,"gram")
    model = NGramModel(i)
    model.fit(seqcorpus)
    print("Predicting...")
    text = " ".join(model.generate_sampled(25,". elle est la plus", power=2))
    print(text)
    print("")

Fitting 2-gram model on vocabulary of size 133657.
Predicting...
. elle est la plus tard , qui est un autre part , le deuxième plus de la région de la fois que les principaux de la langue morte .
cependant

Fitting 3-gram model on vocabulary of size 133657.
Predicting...
. elle est la plus grande partie de la société , la notion de durabilité à l' ouest , le droit de vote de la république , élu président de

Fitting 4-gram model on vocabulary of size 133657.
Predicting...
. elle est la plus grande partie de la population et de l' opportunisme , collection temps critique , 2003 santé & environnement : l' abcdaire , février 2005 j'

Fitting 5-gram model on vocabulary of size 133657.
Predicting...
. elle est la plus grande unité de population au sein de laquelle dl fusionne .
au sein de ce parti , il incarne la ligne libérale en étant membre du



## Beam Search

Remarque : ça se met souvent à boucler ! Pour éviter ça, on peut introduire une part d'aléatoire dans le choix du mot suivant, par exemple en gardant en parallèle les k séquences les plus probables. Cette méthode s'appelle le beam search.

In [17]:
for i in range(2,6):
    # print("Résultats pour",i,"gram")
    model = NGramModel(i)
    model.fit(seqcorpus)
    print("Predicting...")
    text = " ".join(model.generate_beam(25,". elle est la plus", k=3))
    print(text)
    print("")

Fitting 2-gram model on vocabulary of size 133657.
Predicting...
. elle est la plus tard , et de la fin de la fin de la fin de la fin de la fin de la fin de la fin du

Fitting 3-gram model on vocabulary of size 133657.
Predicting...
. elle est la plus grande partie de la ville de caen , par exemple ) , et de la ville de caen , par exemple ) , et le

Fitting 4-gram model on vocabulary of size 133657.
Predicting...
. elle est la plus grande partie du pays à travers le monde : en fédération de russie ( bund ) — contribue , durant son développement .
10 base 5

Fitting 5-gram model on vocabulary of size 133657.
Predicting...
. elle est la plus ancienne forme de chiffrement .
elle permet à la fois de la cueillette terrestre et de la pêche : comme fruits de mer se mangeaient le



Améliorations :
-> lissage (smoothing)
-> Apprendre les fréquences de tous les k-grams pour k<n pour pouvoir switcher à un k plus petit si le k-gram recherché est absent lors de la prédiction
-> Travailler avec les log-probas pour être sûr de ne pas perdre en précision -> Le temps pour générer le texte me semble très long, il y a sans doute moyen d'optimiser le code (optimisation boisseaunienne ou autre)